# Deel Challenge - Raimi Azeez

# Business context
Deel clients may add funds to their Deel account using their credit and debit cards. Deel has
partnered with Globepay to process all of these account funding credit and debit card
transactions. Globepay is an industry-leading global payment processor and is able to process
payments in many currencies from cards domiciled in many countries.
Deel has connectivity into Globepay using their API. Deel clients provide their credit and debit
details within the Deel web application, Deel systems pass those credentials along with any
relevant transaction details to Globepay for processing.

# Problem
Deel is experiencing a decline in the acceptance rate of credit and debit card payments
processed by Globepay in the recent period. **The “acceptance rate” is defined as the number of
accepted transactions divided by the total attempted transactions**

# Root Cause Analysis

The steps for analysing the root cause problem of this project will be as follows:

1. Hypothesis Formulation: Several hypothesis would be formulated the potential root causes. These includes:

  - Currency-conversion issues: Transactions are getting declined due to currency conversion issues.
  - Geographic Factors: Transactions are being declined based on the country they are initiated from.
  - Security Factors: Transactions are getting declined due to lack of extra transaction security, i.e. cvv is not provided.
  - Time Factor: Declined transaction increases during a certain time/period of the year. PS: the project data contains only 6 months of transaction data.
  - Transaction Value: Transactions are getting declined because due to the transaction value (amount).

2. The above listed hypothesis will be investigated using Data Exploration and Visualization techniques (as above).

# Dataset description
- external_ref The card expiry year. Format: 4 digits. For example (_0fqf75KiPa0iiviKCSsU)
- date_time The timestamp of the transaction.
state The binary state of the transaction. For example: Accepted or Declined.
- chargeback If the transaction has been chargedback. For example: True or False
- amount The amount that has been charged from the card.
- currency The three-character ISO currency code.
- country The two-character ISO country code of the card.
- rates The exchange rate used. Funds are settled to you in USD.

===================================================================================================================


# Exploratory Data Analysis

In [1]:
#import packages
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
chargeback = pd.read_csv('chargeback.csv')
chargeback.head()

,external_ref,status,source,chargeback
0,_-UTDS0dMlagMWEtQKMy-,True,GLOBALPAY,False
1,_0fqf75KiPa0iiviKCSsU,True,GLOBALPAY,False
2,_0H9Ecd_RctsLetId-p14,True,GLOBALPAY,False
3,_0wBrMzAzIhO3fQRWUcBH,True,GLOBALPAY,False
4,_25jZuGDrmwUEdAwKp1xV,True,GLOBALPAY,False


In [3]:
chargeback.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5430 entries, 0 to 5429
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   external_ref  5430 non-null   object
 1   status        5430 non-null   bool  
 2   source        5430 non-null   object
 3   chargeback    5430 non-null   bool  
dtypes: bool(2), object(2)
memory usage: 95.6+ KB


In [4]:
chargeback.describe()

,external_ref,status,source,chargeback
count,5430,5430,5430,5430
unique,5430,1,1,2
top,_-UTDS0dMlagMWEtQKMy-,True,GLOBALPAY,False
freq,1,5430,5430,5207


In [5]:
acceptance = pd.read_csv('acceptance.csv', sep=';')
acceptance.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,rates
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."


In [6]:
acceptance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5430 entries, 0 to 5429
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   external_ref  5430 non-null   object 
 1   status        5430 non-null   bool   
 2   source        5430 non-null   object 
 3   ref           5430 non-null   object 
 4   date_time     5430 non-null   object 
 5   state         5430 non-null   object 
 6   cvv_provided  5430 non-null   bool   
 7   amount        5430 non-null   float64
 8   country       5430 non-null   object 
 9   currency      5430 non-null   object 
 10  rates         5430 non-null   object 
dtypes: bool(2), float64(1), object(8)
memory usage: 392.5+ KB


In [7]:
acceptance.describe()

,amount
count,5430.000000
mean,86938.605757
std,50327.853421
min,-23.780000
25%,42481.355000
50%,85488.025000
75%,128548.397500
max,180371.520000


PS: This shows a negative payment (min: -$23.78). Why is there a negative payment made? - Investigate further

How many payments are negative?

In [8]:
neg_payment = acceptance[acceptance['amount'] <= 0]
neg_payment

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,rates
11,SPm_aqm_Rrer_6jxpLvO2,True,GLOBALPAY,evt_1EhCNv4mRDFQzT2r2O5Cy5G,2019-01-01T04:48:00.000Z,ACCEPTED,False,-23.78,UK,GBP,"{""CAD"":1.4173511828553305,""EUR"":0.922579749754..."


There is only one negative payment - This should be investigated further. Is this some kind of Error or Fraudulent transaction??

For the purpose of this analysis, I will drop this transaction.

### Combining both datasets

In [9]:
deel_data = acceptance.merge(chargeback, how='left')
deel_data.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,rates,chargeback
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",False
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",False
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True


In [10]:
#calculate payment amount in USD
amount_usd = []
for index, row in deel_data.iterrows():
  rate = eval(row['rates'])
  usd_rate = round(rate[row['currency']] * row.amount, 2)
  amount_usd.append(usd_rate)

deel_data['amount_USD']  = amount_usd
deel_data.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,rates,chargeback,amount_USD
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",False,1020.46
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True,1582.57
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",False,1653.07
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True,2228.17
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True,2589.92


### To Investigate Further, I Would like to investigate the transactions based on time and of transaction


In [11]:
#convert datetime to local time from utc
deel_data['local_time'] = pd.to_datetime(deel_data['date_time']).dt.tz_localize(None)
deel_data['Year'] = pd.to_datetime(deel_data['local_time']).dt.year
deel_data['Month'] = pd.to_datetime(deel_data['local_time']).dt.month
deel_data['Day'] = pd.to_datetime(deel_data['local_time']).dt.day

deel_data.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,rates,chargeback,amount_USD,local_time,Year,Month,Day
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",False,1020.46,2019-01-01 00:00:00,2019,1,1
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True,1582.57,2019-01-01 04:48:00,2019,1,1
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",False,1653.07,2019-01-01 09:36:00,2019,1,1
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True,2228.17,2019-01-01 14:24:00,2019,1,1
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,...",True,2589.92,2019-01-01 19:12:00,2019,1,1


### Drop rate column (now redundant) & negative payment

In [12]:
#drop negative amount
deel_data.drop(deel_data[deel_data['amount'] < 0].index, inplace = True)

#drop Rate Column
deel_data.drop('rates', axis = 1, inplace = True)

In [13]:
#Final Dataset
deel_data.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,chargeback,amount_USD,local_time,Year,Month,Day
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,False,1020.46,2019-01-01 00:00:00,2019,1,1
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,True,1582.57,2019-01-01 04:48:00,2019,1,1
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,False,1653.07,2019-01-01 09:36:00,2019,1,1
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,True,2228.17,2019-01-01 14:24:00,2019,1,1
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,True,2589.92,2019-01-01 19:12:00,2019,1,1


In [14]:
deel_data.describe(include = 'all')

<ipython-input-14-bcb51cba6a7a>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  deel_data.describe(include = 'all')


,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,chargeback,amount_USD,local_time,Year,Month,Day
count,5429,5429,5429,5429,5429,5429,5429,5429.000000,5429,5429,5429,5.429000e+03,5429,5429.0,5429.000000,5429.000000
unique,5429,1,1,5429,905,2,2,NaN,6,5,2,NaN,905,NaN,NaN,NaN
top,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,ACCEPTED,False,NaN,US,USD,False,NaN,2019-01-01 00:00:00,NaN,NaN,NaN
freq,1,5429,5429,1,6,3758,5389,NaN,905,1810,5207,NaN,6,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-06-30 19:12:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86954.623879,NaN,NaN,NaN,4.289646e+05,NaN,2019.0,3.508749,15.604900
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50318.644430,NaN,NaN,NaN,9.027398e+05,NaN,0.0,1.709936,8.734682
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.190000,NaN,NaN,NaN,3.493000e+01,NaN,2019.0,1.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42486.500000,NaN,NaN,NaN,4.901426e+04,NaN,2019.0,2.000000,8.000000


In [15]:
#deel_data.to_csv('deeldata.csv')

# Visualizations

In [16]:
#declined dataset
deel_decline = deel_data[deel_data['state'] == 'DECLINED']
deel_decline_grouped = deel_decline.groupby('country').sum(numeric_only = True).reset_index()
deel_decline_grouped

,country,status,cvv_provided,amount,chargeback,amount_USD,Year,Month,Day
0,AE,291,0,26335152.43,0,2.633515e+07,587529,1007,4622
1,CA,284,0,26521149.78,0,3.681669e+07,573396,1045,4442
2,FR,280,14,25500758.59,0,1.941610e+07,565320,1007,4336
3,MX,261,0,21970362.99,0,5.509246e+08,526959,899,4068
4,UK,258,0,19713233.74,0,1.420669e+07,520902,862,3985
5,US,297,1,25125669.78,0,2.512567e+07,599643,1019,4760


In [17]:
#plot amount of declined transaction vs country
Country = deel_decline_grouped['country']
Amount_USD = deel_decline_grouped['amount_USD']

fig = go.Figure([go.Bar(x=Country, y=Amount_USD,
                text=Amount_USD,
                texttemplate='%{text:.2s}', textposition='outside',
                        marker_color='red')])
fig.update_layout(title_text='Country vs Amount(USD) - DECLINED TRANSACTIONS',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=800,
                  height=500,
                  yaxis=dict(
                      title_text="Amount(USD)",
                      ),
                  xaxis=dict(
                      title_text="Country",
                      )
                  )

fig.show()

This shows a significant amount of decline rate in MX compared to other countries. Why? First, let's look compare the number of approved transactions to declined transactions

In [18]:
#accepted dataset
deel_accept = deel_data[deel_data['state'] == 'ACCEPTED']
deel_accept_grouped = deel_accept.groupby('country').sum(numeric_only = True).reset_index()

#select columns to plot for declined and accepted transactions
country_acpt = deel_accept_grouped['country']
amount_USD_acpt = deel_accept_grouped['amount_USD']

country_dec = deel_decline_grouped['country']
amount_USD_dec = deel_decline_grouped['amount_USD']


#plot
fig = go.Figure([
    go.Bar(name='Declined', x=country_dec, y=amount_USD_dec,
            text=amount_USD_dec,
            texttemplate='%{text:.2s}', textposition='outside',
            marker_color='red'),
    go.Bar(name='Accepted', x=country_acpt, y=amount_USD_acpt,
            text=amount_USD_acpt,
            texttemplate='%{text:.3s}', textposition='outside',
           marker_color='green')
    ])
fig.update_layout(barmode='group',
                  title_text='Country vs Amount(USD)',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=900,
                  height=500,
                  yaxis=dict(
                      title_text="Amount(USD)",
                      ),
                  xaxis=dict(
                      title_text="Country",
                      )
                  )

fig.show()

This brings into perspective that the amount(USD) of declined transactions from MX is not so significant when compared to the amount of accepted transactions. Investigate further

- What is the acceptance rate (decline rate) for each country?



Let's calculate acceptance rates based on data variables

In [19]:
#define function to calculate acceptance rate based on variable
def calc_acceptance_rate(df, column):
  """
  :param df(DataFrame): transaction dataset to calculate acceptance rate.
  :param column(string): column to calculate acceptance rate for
  :return(list): The acceptance rate of specified culumn.
  """
  accept_rate = []
  for i in list(df[column].unique()):
    accept_count = len(df.loc[(df[column] == i) & (df['state'] == 'ACCEPTED')])
    total_count = len(df.loc[(df[column] == i)])

    rate = round(accept_count / total_count, 2)
    accept_rate.append(rate)

  return accept_rate


In [20]:
deel_data['country'].unique()

array(['US', 'MX', 'UK', 'FR', 'CA', 'AE'], dtype=object)

In [21]:

country_accept_rate = calc_acceptance_rate(deel_data, 'country')

#plot

fig = go.Figure([
    go.Bar(name='Declined', y=country_accept_rate, x=deel_data['country'].unique(),
            text=country_accept_rate,
            textposition='outside',
            marker_color='green'),

    ])
fig.update_layout(barmode='group',
                  title_text='Country Acceptance Rate ',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=900,
                  height=500,
                  yaxis=dict(
                      title_text="Acceptance Rate",
                      ),
                  xaxis=dict(
                      title_text="Country",
                      )
                  )

fig.show()

In [22]:
currency_accept_rate = calc_acceptance_rate(deel_data, 'currency')

#plot
fig = go.Figure([
    go.Bar(name='Declined', y=currency_accept_rate, x=deel_data['currency'].unique(),
            text=currency_accept_rate,
            textposition='outside',
            marker_color='green'),

    ])
fig.update_layout(barmode='group',
                  title_text='Currency Acceptance Rate ',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=900,
                  height=500,
                  yaxis=dict(
                      title_text="Acceptance Rate",
                      ),
                  xaxis=dict(
                      title_text="Currency",
                      )
                  )

fig.show()

Safe to say The acceptance rate is not dependent of the customer's currency or country as there seem to be no significant difference between the acceptance raate of the currencies and countries.

Let's look at Transactions where CVV was provided.

In [23]:
#Acceptance rate for transaction where cvv was provided
deel_ccv = deel_data[deel_data['cvv_provided'] == True]

deel_cvv_accepted = deel_ccv[deel_ccv['state'] == 'ACCEPTED']
cvv_acceptance_rate = round(len(deel_cvv_accepted)/(len(deel_ccv)), 2)
print(f'Acceptance Rate for transactions where CVV was provided: {cvv_acceptance_rate}')

#Acceptance rate for transaction where cvv was not provided
deel_ccv = deel_data[deel_data['cvv_provided'] == False]

deel_cvv_accepted = deel_ccv[deel_ccv['state'] == 'ACCEPTED']
no_cvv_acceptance_rate = round(len(deel_cvv_accepted)/(len(deel_ccv)), 2)
print(f'Acceptance Rate for transactions where CVV was not provided: {no_cvv_acceptance_rate}')

Acceptance Rate for transactions where CVV was provided: 0.62
Acceptance Rate for transactions where CVV was not provided: 0.69


In [24]:
#plot
cvv = [no_cvv_acceptance_rate, cvv_acceptance_rate]
fig = go.Figure([
    go.Bar(name='Declined', y=cvv, x=deel_data['cvv_provided'].unique(),
            text=cvv,
            textposition='outside',
            marker_color='green'),

    ])
fig.update_layout(barmode='group',
                  title_text='CVV Provided Acceptance Rate ',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=500,
                  height=500,
                  yaxis=dict(
                      title_text="Acceptance Rate",
                      ),
                  xaxis=dict(
                      title_text="CVV Provided",
                      )
                  )

fig.show()

Let's look at the acceptance rate by month of the year


In [25]:
#accepted dataset
deel_month = deel_data[deel_data['state'] == 'ACCEPTED']
deel_month_grouped = deel_month.groupby('Month').sum(numeric_only=True).reset_index()
month_acpt = deel_month_grouped['Month']
amount_USD_acpt = deel_month_grouped['amount_USD']

#declined dataset
deel_month_dec = deel_data[deel_data['state'] == 'DECLINED']
deel_month_grouped_dec = deel_month_dec.groupby('Month').sum(numeric_only=True).reset_index()
month_dec = deel_month_grouped_dec['Month']
amount_USD_dec = deel_month_grouped_dec['amount_USD']


#plot
fig = go.Figure([
    go.Bar(name='Declined', x=month_dec, y=amount_USD_dec,
            text=amount_USD_dec,
            texttemplate='%{text:.2s}', textposition='outside',
            marker_color='red'),
    go.Bar(name='Accepted', x=month_acpt, y=amount_USD_acpt,
            text=amount_USD_acpt,
            texttemplate='%{text:.3s}', textposition='outside',
           marker_color='green')
    ])
fig.update_layout(barmode='group',
                  title_text='Month vs Amount(USD)',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=900,
                  height=500,
                  yaxis=dict(
                      title_text="Amount(USD)",
                      ),
                  xaxis=dict(
                      title_text="Month",
                      )
                  )

fig.show()

In [26]:
month_accept_rate = calc_acceptance_rate(deel_data, 'Month')

#plot
fig = go.Figure([
    go.Bar(x=deel_data['Month'].unique(), y=month_accept_rate,
            text=month_accept_rate,
           textposition='outside',
            marker_color='green'),
    ])
fig.update_layout(
                  title_text='Monthly Acceptance Rate',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=900,
                  height=500,
                  yaxis=dict(
                      title_text="Acceptance Rate",
                      ),
                  xaxis=dict(
                      title_text="Month",
                      )
                  )

fig.show()

Let's Investigate Sales Amount(USD) vs Acceptance Rate

In [27]:
#Create a bin for sales amount

deel_data['amount_bin_USD'] = pd.qcut(deel_data['amount_USD'], 5)
deel_data.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,chargeback,amount_USD,local_time,Year,Month,Day,amount_bin_USD
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,False,1020.46,2019-01-01 00:00:00,2019,1,1,"(34.929, 39256.308]"
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,True,1582.57,2019-01-01 04:48:00,2019,1,1,"(34.929, 39256.308]"
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,False,1653.07,2019-01-01 09:36:00,2019,1,1,"(34.929, 39256.308]"
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,True,2228.17,2019-01-01 14:24:00,2019,1,1,"(34.929, 39256.308]"
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,True,2589.92,2019-01-01 19:12:00,2019,1,1,"(34.929, 39256.308]"


In [28]:
amount_accept_rate = calc_acceptance_rate(deel_data, 'amount_bin_USD')

#plot
fig = go.Figure([
    go.Bar(x=deel_data['amount_bin_USD'].unique().astype(str), y=amount_accept_rate,
            text=amount_accept_rate,
           textposition='outside',
            marker_color='green'),
    ])
fig.update_layout(
                  title_text='Amount(USD) Acceptance Rate',
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=900,
                  height=500,
                  yaxis=dict(
                      title_text="Acceptance Rate",
                      ),
                  xaxis=dict(
                      title_text="Amount(USD)",
                      )
                  )

fig.show()

# Task #2
Working on the same dataset, please provide pandas code to answer the following questions:
- What is the acceptance rate over time?
- List the countries where the amount of declined transactions went over $25M
- Which transactions are missing chargeback data?

### Acceptance Rate Overtime

In [29]:
#Total Acceptance Rate
def acceptance_rate(df):
  """
  :param df(DataFrame): transaction dataset to calculate acceptance rate.
  :return(float): The total acceptance rate of the dataset.
  """

  no_accepted = len(df[df['state'] == 'ACCEPTED'])

  return round(no_accepted/len(df), 2)

In [30]:
print(f'Acceptance Rate: {acceptance_rate(deel_data)}')

Acceptance Rate: 0.69


In [31]:
for mon, acc in zip(deel_data['Month'].unique(), month_accept_rate):
  print(f'Acceptance Rate for Month {mon}: {acc}')

Acceptance Rate for Month 1: 0.7
Acceptance Rate for Month 2: 0.7
Acceptance Rate for Month 3: 0.69
Acceptance Rate for Month 4: 0.66
Acceptance Rate for Month 5: 0.69
Acceptance Rate for Month 6: 0.72


### Declined transactions above over $25M

In [32]:
def country_over(df, amount):
  """
  :param df(DataFrame): transaction dataset to calculate decline volume
  :param amount(float): amount(USD) to check if decline volume is over
  :return(list): The countries which declined transaction volumne is greater than amount specified.
  """
  df = df[df['state'] == 'DECLINED']
  df = df.groupby('country').sum(numeric_only = True).reset_index()
  countries = list(df[df['amount_USD'] > amount]['country'].unique())

  if len(countries) == 0:
    countries = 'None'

  return countries

In [33]:
print(f'Countries where amount of declined transactions went over $25M: {country_over(deel_data, 25000000)}')

Countries where amount of declined transactions went over $25M: ['AE', 'CA', 'MX', 'US']


### Missing chargeback data

In [34]:
null_chargeback = deel_data[deel_data['chargeback'].isna()] #.sum()

if len(null_chargeback) == 0:
  null_chargeback = None


null_chargeback
print(f'Transactions missing chargeback data: {null_chargeback}')

Transactions missing chargeback data: None


# Task #1
Please present a report (presentation/keynote/pdf) that:
- Outlines the volume (in USD) of the declined payments
- Analyses the problem root causes
- Suggests solutions / recommendations
- Lists further next steps (hypothesis / analysis) you would do as a next steps having more
time and business knowledge

In [35]:
decline_vol = deel_decline['amount_USD'].sum()

print(f'Volume (USD) of declined payments: {"${:,.2f}M".format(decline_vol/ 1000000)}')

Volume (USD) of declined payments: $672.82M


In [36]:
deel_decline_curr = deel_data[deel_data['state'] == 'DECLINED']
deel_decline_curr = deel_decline_curr.groupby('currency').sum(numeric_only = True).reset_index()
deel_decline_curr['Total Decline Volume'] = decline_vol
fig = px.bar(deel_decline_curr, x='Total Decline Volume', y='amount_USD',color='currency')

fig.update_xaxes(showticklabels=False, title=None)
fig.update_traces(width=.3)
fig.update_layout(
                  title_text=f'TOTAL DECLINE VOLUME: {"${:,.2f}M".format(decline_vol/ 1000000)}', #"{:,. 2f}M"
                  margin=dict(l=50, r=20, t=50, b=50),
                  width=700,
                  height=500,
                  yaxis=dict(
                      title_text="Acceptance Rate",
                      ),
                  # xaxis=dict(
                  #     title_text="Total Declined Amount",
                  #     )
                  )
fig.show()

## Findings

The Investigations done reveiled that none of the above listed hypothesis was true. All factors investigated shows no significant relationship with the acceptance rate of Transactions.

Note:
- Investigate negative payment
- Investigate accepted payment where chargback is True



## Recommendations

- Customer Support: Declined transaction will negatively impact customer experience, having a responsive customer support will help build customer trust and confidence for continuous patronage.

- Contact GLOBEPAY to find out their transaction processing methods (fraud detection, currency conversion processes) and suggest/request business-specific processing methods for Deel to improve acceptance rate.
  - Are the declined transactions suspected fraudulent transactions?
  - How is the currency conversion done?

- Contract an alternative payment processing company: The company is losing a significant amount of money to declined transactions ($672M in 6 months) which is definetly bad for business and user experience. Having an alternative payment processing option will give customers option to use a different service, and also provide data for benchmarking.




## Next steps

Given more data and better business knowledge, i'll suggest the following:

- A/B Testing: I strongly recommend some statistical tests to compare the performcance of GLOBEPAY with industry benchmark (if data is available), and also do some significance tests with every changes made to the payment processing process to measure their impact and learn where to focus on for more improvement.

- Customer Segmentation: Given information available on customers, a customer segmentation model could provide more insight on common features of declined transactions.

- Investigate Technical Compatibility: Investigate the software development and API connection process to ensure standard practices and ensure transaction security.

- Client Feedback: Collect client feedback on every transaction to get more insight on their experience

End! Thank you!